In [1]:
import numpy as np
import pandas as pd
import torch
from pypots.utils.metrics.error import calc_mae, calc_rmse, calc_mre

import sys
sys.path.append("../scripts")
from utils import DataReader, SynDataset
from train import BTSTN

In [2]:
########################################
# Data
########################################
train_file = "../data/simulated_p53_mdm2/simulated_p53_mdm2_sustained.csv"
train_data = pd.read_csv(train_file)
train_data = train_data.iloc[:2000, :]
train_dr = DataReader(data=train_data, scaler="MinMax", dropna=False, unit=0.1)

In [3]:
########################################
# Parameters
########################################
n_features = 2
n_dims = 16
g_inner = 32
g_layers = 2
dropout = 0.1
max_gap = 10

In [4]:
########################################
# Initialize the model
########################################
tcnits = BTSTN(
    n_features=n_features,
    n_dims=n_dims,
    g_inner=g_inner,
    g_layers=g_layers,
    d_inner=None,
    d_layers=None,
    dropout=dropout,
    d_dropout=0,
    activation={"fc_name":"tanh"},
    max_gap=max_gap,
    batch_size=128,
    epoch=1000,
    patience=30,
    learning_rate=0.01,
    threshold=0,
    gpu_id=-1,
    num_workers=0,
    pin_memory=False,
    saving_path=None,
    saving_prefix=None
)

In [ ]:
print("Sustained dataset")
for n in range(3):
    ########################################
    # Train
    ########################################
    train_loss = tcnits.fit(data=train_dr)

    ########################################
    # Reconstruction
    ########################################
    device = torch.device("cpu")
    inverse = True
    f_module = tcnits.model["F"].to(device)
    d_module = tcnits.model["D"].to(device)
    unit = torch.FloatTensor(train_dr.unit_mx)
    unit = unit.to(device)
    fdata = f_module(unit)
    output = d_module(fdata)
    output = output.detach().to("cpu").numpy()
    if inverse and train_dr.scaler is not None:
        output = train_dr.scaler.inverse_transform(output)

    ########################################
    # Evaluation of Missing pattern
    ########################################
    miss_mask_00 = np.ones_like(output)
    miss_mask_01 = miss_mask_00.copy()
    miss_mask_01[:, 0] = 0
    miss_mask_02 = miss_mask_00.copy()
    miss_mask_02[:, 1] = 0
    true = np.array(train_dr.dataframe)[:, 2:]

    mae_00 = calc_mae(output, true, miss_mask_00)
    rmse_00 = calc_rmse(output, true, miss_mask_00)
    mre_00 = calc_mre(output, true, miss_mask_00)

    mae_01 = calc_mae(output, true, miss_mask_01)
    rmse_01 = calc_rmse(output, true, miss_mask_01)
    mre_01 = calc_mre(output, true, miss_mask_01)

    mae_02 = calc_mae(output, true, miss_mask_02)
    rmse_02 = calc_rmse(output, true, miss_mask_02)
    mre_02 = calc_mre(output, true, miss_mask_02)

    print("\nReplicate: {}".format(n))
    print("MAE:", mae_00, "MAE_Mdm2:", mae_01, "MAE_P53:", mae_02)
    print("RMSE:", rmse_00, "RMSE_Mdm2:", rmse_01, "RMSE_P53:", rmse_02)
    print("MRE:", mre_00, "MRE_Mdm2:", mre_01, "MRE_P53:", mre_02)